In [14]:
import tensorflow as tf

# for example in tf.python_io.tf_record_iterator("./card-train.record"):
#     print(tf.train.Example.FromString(example))
#     break
for example in tf.python_io.tf_record_iterator("./bdd100k/tfrecord/100k/val/output_val100k__000000.tfrecord"):
    print(tf.train.Example.FromString(example))
    break


features {
  feature {
    key: "image/encoded"
    value {
      bytes_list {
        value: "\377\330\377\376\000\020Lavc56.60.100\000\377\333\000C\000\010\016\016\020\016\020\023\023\023\023\023\023\026\025\026\027\027\027\026\026\026\026\027\027\027\031\031\031\035\035\035\031\031\031\027\027\031\031\034\034\035\035 ! \036\036\035\036!!###**((112<<H\377\304\001\242\000\000\001\005\001\001\001\001\001\001\000\000\000\000\000\000\000\000\001\002\003\004\005\006\007\010\t\n\013\001\000\003\001\001\001\001\001\001\001\001\001\000\000\000\000\000\000\001\002\003\004\005\006\007\010\t\n\013\020\000\002\001\003\003\002\004\003\005\005\004\004\000\000\001}\001\002\003\000\004\021\005\022!1A\006\023Qa\007\"q\0242\201\221\241\010#B\261\301\025R\321\360$3br\202\t\n\026\027\030\031\032%&\'()*456789:CDEFGHIJSTUVWXYZcdefghijstuvwxyz\203\204\205\206\207\210\211\212\222\223\224\225\226\227\230\231\232\242\243\244\245\246\247\250\251\252\262\263\264\265\266\267\270\271\272\302\303\304\305\306\307\3

In [12]:
import argparse

import matplotlib.pyplot as plt
import tensorflow as tf
from matplotlib.patches import Rectangle

from deepdrive_dataset.deepdrive_dataset_reader import DeepdriveDatasetReader
from deepdrive_dataset.deepdrive_versions import DEEPDRIVE_FOLDS, DEEPDRIVE_VERSIONS


# parser = argparse.ArgumentParser()
# parser.add_argument('--batch_size', type=int, help='Batch Size to use', default=4)
# parser.add_argument('--fold_type', type=str, choices=DEEPDRIVE_FOLDS, default='train')
# parser.add_argument('--version', type=str, default='100k', choices=DEEPDRIVE_VERSIONS)
batch_size = 4
version = 'train'
fold_type='100k'

reader = DeepdriveDatasetReader(batch_size=batch_size)
if fold_type == 'train':
    iterator = reader.load_train_data_bbox(version, False)
elif fold_type == 'val':
    iterator = reader.load_val_data_bbox(version, False)
elif fold_type == 'test':
    iterator = reader.load_test_data_bbox(version, False)
else:
    raise BaseException('Unknown fold type: {0}'.format(fold_type))

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    out = sess.run(iterator)
    out_labels = DeepdriveDatasetReader.parsing_boundingboxes(None, 'labels')
    out_dict = dict(zip(out_labels, out))

    # Visualize the output
    import math
    images = out_dict['image']
    fig_row = int(math.ceil(images.shape[0] / 2))
    fig_col = int(math.ceil(images.shape[0] / 2))
    f, axarr = plt.subplots(nrows=fig_row, ncols=fig_col)
    for im_id in range(0, images.shape[0]):
        plt.sca(axarr[im_id % 2, int(im_id/2)])
        plt.imshow(images[im_id, :])
        # Boundingbox order is ymin, xmin, ymax, xmax
        for box in out_dict['bboxes'][im_id, :]:
            rect = Rectangle(
                xy=(box[1], box[0]), width=box[3] - box[1], height=box[2] - box[0], fill=False)
            axarr[im_id % 2, int(im_id/2)].add_patch(rect)

        #
        axarr[im_id % 2, int(im_id/2)].set_xticks([])
        axarr[im_id % 2, int(im_id/2)].set_yticks([])
        axarr[im_id % 2, int(im_id/2)].set_ylabel('Batch-Id: {0}'.format(im_id))
        axarr[im_id % 2, int(im_id/2)].set_xlabel(str(out_dict['image_ids'][im_id]))
    plt.show()



TFRecord path does not exists: /home/bokang.jia/.deepdrive/tfrecord. First create the tfrecord file.


NameError: name 'exit' is not defined